In [1]:
# IMPORTS
import tensorflow
# print(tensorflow.version.VERSION)

import sys, os
# Important to assign the processing to the GPU with the most vRAM available
os.environ["CUDA_VISIBLE_DEVICES"]="1" #"1"

physical_devices = tensorflow.config.experimental.list_physical_devices('GPU')

config = tensorflow.compat.v1.ConfigProto(device_count = {'GPU': 1})

#Important to run this in order to not overload the GPU
config = tensorflow.compat.v1.ConfigProto(log_device_placement=True)
config.gpu_options.per_process_gpu_memory_fraction=0.9 # don't hog all vRAM
#config.operation_timeout_in_ms=15000   # terminate on long hangs
#config.gpu_options.allow_growth = True
sess = tensorflow.compat.v1.InteractiveSession("", config=config)

import glob
import numpy as np
import nibabel as nib
import pandas as pd
import random
import time
import matplotlib.pyplot as plt
import random

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Tesla T4, pci bus id: 0000:00:0b.0, compute capability: 7.5



In [2]:
# THIS SECTION CONCATENATES THE TRAIN_EVEN ARRAYS AND TEST_ODD ARRAYS SEPERATELY
# SO THAT THEY CAN BE JOINED TOGETHER AFTER
# READ THE NEXT CELL FOR DETAILED COMMENTS OF CODE STRUCTURE
organ = 'PAROTID_LT' #BRAIN_STEM, SPINAL_CORD, PAROTID_RT
train_test = 'train_odd' # test_even train_odd
folder_arrays = '/hepgpu9-data1/sbridger/mphys_2d_network_2021-22/slice_arrays/new_registrations/' + organ + '/' + train_test + '/'
print(folder_arrays)

def filepath_length(fp):
    return len(glob.glob(fp+'*/'))

registration_filepath = '/hepgpu9-data1/sbridger/Auto_Registration/registration_store/new_registration/' + train_test + '/' 
n_reg = filepath_length(registration_filepath)
print(n_reg)
indexes=[]
# step_size=545 # KEEP THIS NUMBER DEPENDING ON HOW YOU RAN THE Load_2D_Slices...selection CODE
# for i in range(0, n_reg, step_size):
#     if i+step_size-1 > n_reg:
#         indexes.append(str(i) + '-' + str(n_reg-1))
#     else:
#         indexes.append(str(i) + '-' + str(i+step_size-1))
if train_test=='train_odd':
    indexes.append('0-544')
    indexes.append('545-1033')
else:
    indexes.append('0-544')
    indexes.append('545-1080')
    
idxs=len(indexes)
image_array_name, ref_contour_array_name, def_contour_array_name, dice_array_name, hd_95_array_name, hd_75_array_name,\
hd_50_array_name, rms_array_name, msd_array_name, registration_track_name = [], [], [], [], [], [], [], [], [], []
for i in range(idxs):
    suffix = train_test + '_' + organ + '_' + str(indexes[i]) + '.npy'
#     image_array_name.append(folder_arrays + 'image_dvf_array_' + suffix)
#     ref_contour_array_name.append(folder_arrays + 'ref_contour_image_array_' + suffix)
    def_contour_array_name.append(folder_arrays + 'def_contour_image_array_' + suffix)

#     dice_array_name.append(folder_arrays + 'dice_array_' + suffix)
#     hd_95_array_name.append(folder_arrays + 'hd_95_array_' + suffix)
#     hd_75_array_name.append(folder_arrays + 'hd_75_array_' + suffix)
#     hd_50_array_name.append(folder_arrays + 'hd_50_array_' + suffix)
#     rms_array_name.append(folder_arrays + 'rms_array_' + suffix)
#     msd_array_name.append(folder_arrays + 'msd_array_' + suffix)
#     registration_track_name.append(folder_arrays + 'registration_track_' + suffix)

mpia, ref_ctour, def_ctour, mhd95, mhd75, mhd50, mrms, mda, msd, rt = [], [], [], [], [], [], [], [], [], []

for i in range(idxs):
#     ref_ctour.append(np.load(ref_contour_array_name[i], mmap_mode = 'r'))
    
    def_ctour.append(np.load(def_contour_array_name[i], mmap_mode = 'r'))
    
#     mpia.append(np.load(image_array_name[i], mmap_mode = 'r'))

#     mda.append(np.load(dice_array_name[i], mmap_mode = 'r'))
    
#     mhd95.append(np.load(hd_95_array_name[i], mmap_mode = 'r'))
    
#     mhd75.append(np.load(hd_75_array_name[i], mmap_mode = 'r'))
    
#     mhd50.append(np.load(hd_50_array_name[i], mmap_mode = 'r'))
    
#     mrms.append(np.load(rms_array_name[i], mmap_mode = 'r'))
    
#     msd.append(np.load(msd_array_name[i], mmap_mode = 'r'))
        
#     rt.append(np.load(registration_track_name[i], mmap_mode = 'r'))

# print(len(multiple_patient_image_array), len(multiple_dice_array), len(multiple_hd_95_array_3D), len(multiple_msd_array_3D))
# registration_track is of shape [flo_no, ref_no, len(multiple_msd_array_3D), patient_image_array.shape[0]])

def_contour_image_length, image_array_length, metric_array_length, rt_length = 0, 0, 0, 0
for i in range(idxs):
    def_contour_image_length+=len(def_ctour[i])
#     image_array_length+=len(mpia[i])
#     metric_array_length+=len(mda[i])
#     rt_length+=len(rt[i])

print("def contour array length: ", def_contour_image_length)
# print("image array length: ", image_array_length)
# print("metric array length: ", metric_array_length) 
# print("registration track length: ", rt_length) 


/hepgpu9-data1/sbridger/mphys_2d_network_2021-22/slice_arrays/new_registrations/PAROTID_LT/train_odd/
1081
def contour array length:  14353


In [3]:
def fpl_arr(arr, idx):
    return len(arr[idx])

folder = '/hepgpu9-data1/sbridger/mphys_2d_network_2021-22/slice_arrays/new_registrations/'+organ+'/complete_arrays/'
prefix = train_test + '_' + organ + '_'
data_type = np.float16
tmp_contour_arr = np.empty((def_contour_image_length,512,512), dtype=data_type)
tmp_img_arr = np.empty((image_array_length,512,512,4), dtype=data_type)
tmp_metric_arr = np.empty(metric_array_length, dtype=data_type)
tmp_reg_track = np.empty((rt_length,4), dtype=data_type)

def_contour_image_arrayT, ref_contour_image_arrayT, multiple_patient_image_arrayT, multiple_hd_95_arrayT, multiple_hd_75_arrayT,\
multiple_hd_50_arrayT, multiple_dice_arrayT, multiple_rms_arrayT, multiple_msd_arrayT, registration_trackT\
= tmp_contour_arr, tmp_contour_arr, tmp_img_arr, tmp_metric_arr, tmp_metric_arr, tmp_metric_arr, tmp_metric_arr, tmp_metric_arr,\
tmp_metric_arr, tmp_reg_track
print('here')
complete_def_contour_name = folder+prefix+'multiple_def_contour_image_array.npy'
complete_ref_contour_name = folder+prefix+'multiple_ref_contour_image_array.npy'
complete_img_name = folder+prefix+'multiple_patient_image_dvf_array.npy'
complete_dice_name = folder+prefix+'multiple_dice_array.npy'
complete_hd_95_name = folder+prefix+'multiple_hd_95_array.npy'
complete_hd_75_name = folder+prefix+'multiple_hd_75_array.npy'
complete_hd_50_name = folder+prefix+'multiple_hd_50_array.npy'
complete_rms_name = folder+prefix+'multiple_rms_array.npy'
complete_msd_name = folder+prefix+'multiple_msd_array.npy'
complete_reg_track_name = folder+prefix+'registration_track.npy'


names = [complete_img_name, complete_dice_name, complete_hd_95_name, complete_hd_75_name,complete_hd_50_name,\
         complete_rms_name, complete_msd_name, complete_reg_track_name, complete_ref_contour_name,\
        complete_def_contour_name]

# np.save(complete_ref_contour_name, ref_contour_image_arrayT)
np.save(complete_def_contour_name, def_contour_image_arrayT)
# np.save(complete_img_name, multiple_patient_image_arrayT)
# np.save(complete_dice_name, multiple_dice_arrayT)
# np.save(complete_hd_95_name, multiple_hd_95_arrayT)
# np.save(complete_hd_75_name, multiple_hd_75_arrayT)
# np.save(complete_hd_50_name, multiple_hd_50_arrayT)
# np.save(complete_rms_name, multiple_rms_arrayT)
# np.save(complete_msd_name, multiple_msd_arrayT)
# np.save(complete_reg_track_name, registration_trackT)

print('done')

# multiple_patient_image_array = np.lib.format.open_memmap(names[0], dtype=data_type, mode='r+', shape=tmp_img_arr.shape)
# multiple_dice_array = np.lib.format.open_memmap(names[1], dtype=data_type, mode='r+', shape=tmp_metric_arr.shape)
# multiple_hd_95_array = np.lib.format.open_memmap(names[2], dtype=data_type, mode='r+', shape=tmp_metric_arr.shape)
# multiple_hd_75_array = np.lib.format.open_memmap(names[3], dtype=data_type, mode='r+', shape=tmp_metric_arr.shape)
# multiple_hd_50_array = np.lib.format.open_memmap(names[4], dtype=data_type, mode='r+', shape=tmp_metric_arr.shape)
# multiple_rms_array = np.lib.format.open_memmap(names[5], dtype=data_type, mode='r+', shape=tmp_metric_arr.shape)
# multiple_msd_array = np.lib.format.open_memmap(names[6], dtype=data_type, mode='r+', shape=tmp_metric_arr.shape)
# multiple_registration_track = np.lib.format.open_memmap(names[7], dtype=data_type, mode='r+', shape=(tmp_reg_track.shape))
# multiple_ref_contour_array = np.lib.format.open_memmap(names[8], dtype=data_type, mode='r+', shape=tmp_contour_arr.shape)
multiple_def_contour_array = np.lib.format.open_memmap(names[9], dtype=data_type, mode='r+', shape=tmp_contour_arr.shape)


def concatenate_memmap_metric_arrays(empty_array, value_array):
    first_arr=fpl_arr(value_array,0)
    second_arr=fpl_arr(value_array,1)
#     print(first_arr,second_arr)
    for n in range(first_arr):
        empty_array[n] = value_array[0][n]
    for n in range(second_arr):
        empty_array[n+first_arr] = value_array[1][n]


def concatenate_memmap_image_arrays(empty_array, value_array):
    first_arr=fpl_arr(value_array,0)
    second_arr=fpl_arr(value_array,1)
#     print(first_arr,second_arr)
    for n in range(first_arr):        
        empty_array[n][:][:][:] = value_array[0][n][:][:][:]
        if n%100==0:
            print(n)
    for n in range(second_arr):
        empty_array[n+first_arr][:][:][:] = value_array[1][n][:][:][:] 
        if n%100==0:
            print(n)
            
# concatenate_memmap_image_arrays(multiple_patient_image_array, mpia)
# concatenate_memmap_metric_arrays(multiple_dice_array, mda)
# concatenate_memmap_metric_arrays(multiple_hd_95_array, mhd95)
# concatenate_memmap_metric_arrays(multiple_hd_75_array, mhd75)
# concatenate_memmap_metric_arrays(multiple_hd_50_array, mhd50)
# concatenate_memmap_metric_arrays(multiple_rms_array, mrms)
# concatenate_memmap_metric_arrays(multiple_msd_array, msd)
# concatenate_memmap_metric_arrays(multiple_registration_track, rt)
# concatenate_memmap_image_arrays(multiple_ref_contour_array, ref_ctour)
concatenate_memmap_image_arrays(multiple_def_contour_array, def_ctour)
                
# print(multiple_patient_image_array.shape)

# diffhd, diffmsd, diffd =[],[],[]
# for i in range(20):
#     diffhd.append(multiple_hd_95_array[i]-mhd95[0][i])
#     diffd.append(multiple_dice_array[i]-mda[0][i])

# print(np.max(diffhd), np.max(diffd))

# MEMMAP ARRAYS MUST BE FLUSHED TO SAVE THE ARRAYS TO DISK
# multiple_patient_image_array.flush()
# multiple_dice_array.flush()
# multiple_hd_95_array.flush()
# multiple_hd_75_array.flush()
# multiple_hd_50_array.flush()
# multiple_rms_array.flush()
# multiple_msd_array.flush()
# multiple_registration_track.flush()
# multiple_ref_contour_array.flush()
multiple_def_contour_array.flush()
                                                        

here
done
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100


In [ ]:
# THIS SECTION CONCATENATES THE TRAIN_EVEN AND TEST_ODD ARRAYS
# INTO THE FULL, COMPLETE ARRAYS TO BE USED ON THE CNN.
# YOU CAN'T RUN BOTH SECTIONS ON CONCATENATION ONE AFTER THE OTHER DUE TO 
# VARIABLE NAMES CHOSEN I.E. YOU CAN ONLY RUN ONE SECTION AT ONCE 
# OTHERWISE OVERWRITING WILL OCCUR.

# 'BRAIN_STEM', 'SPINAL_CORD', 'PAROTID_RT', 'PAROTID_LT' 
organ = 'PAROTID_LT'
machine = 'hepgpu9'
train_test_list = ['train_odd','test_even']
data_type = np.float16
# train_test = 'test_odd' # train_even
train_test_complete_img_name, train_test_complete_dice_name,\
train_test_complete_hd_95_name, train_test_complete_hd_75_name, train_test_complete_hd_50_name,\
train_test_complete_rms_name, train_test_complete_msd_name,\
train_test_complete_msd_3d_name, train_test_complete_registration_track_name = [], [], [], [], [], [], [], [], []

folder = '/hepgpu9-data1/sbridger/mphys_2d_network_2021-22/slice_arrays/new_registrations/' +\
organ + '/complete_arrays/'
for train_test in train_test_list:
    prefix = train_test + '_' + organ + '_'
    train_test_complete_img_name.append(folder+prefix+'multiple_patient_image_array.npy')
    train_test_complete_dice_name.append(folder+prefix+'multiple_dice_array.npy')
    train_test_complete_hd_95_name.append(folder+prefix+'multiple_hd_95_array.npy')
    train_test_complete_hd_75_name.append(folder+prefix+'multiple_hd_75_array.npy')
    train_test_complete_hd_50_name.append(folder+prefix+'multiple_hd_50_array.npy')
    train_test_complete_rms_name.append(folder+prefix+'multiple_rms_array.npy')
    train_test_complete_msd_name.append(folder+prefix+'multiple_msd_array.npy')
    train_test_complete_registration_track_name.append(folder+prefix+'registration_track.npy')

# print(train_test_complete_img_name)
# print(train_test_complete_dice_name)
print(train_test_complete_registration_track_name)

idxs=len(train_test_complete_img_name)
mpia, mda, mhd95, mhd75, mhd50, mrms, msd, rt = [], [], [], [], [], [], [], [], []
start = time.time()
for i in range(idxs):
#     mpia.append(np.load(train_test_complete_img_name[i], mmap_mode = 'r'))

    mda.append(np.load(train_test_complete_dice_name[i], mmap_mode = 'r'))
    
    mhd95.append(np.load(train_test_complete_hd_95_name[i], mmap_mode = 'r'))
    
    mhd75.append(np.load(train_test_complete_hd_75_name[i], mmap_mode = 'r'))
    
    mhd50.append(np.load(train_test_complete_hd_50_name[i], mmap_mode = 'r'))
    
    mrms.append(np.load(train_test_complete_rms_name[i], mmap_mode = 'r'))
    
    msd.append(np.load(train_test_complete_msd_name[i], mmap_mode = 'r'))
        
    rt.append(np.load(train_test_complete_registration_track_name[i], mmap_mode = 'r'))


stop = time.time()
print("Loading the arrays took: " + str(stop-start) + " seconds.")

# multiple_patient_image_array
# print(len(mpia))
# print(type(mpia[0][0][0][0][0]))
# print(len(multiple_patient_image_array), len(multiple_dice_array), len(multiple_hd_95_array_3D), len(multiple_msd_array_3D))

# registration_track is of shape [flo_no, ref_no, len(multiple_msd_array_3D), patient_image_array.shape[0]])

image_array_length, metric_array_length, rt_length = 0, 0, 0
for i in range(idxs):
#     image_array_length+=len(mpia[i])
    metric_array_length+=len(mda[i])
    rt_length+=len(rt[i])

# print("image array length: ", image_array_length)
print("metric array length: ", metric_array_length) 
print("registration track length: ", rt_length) 

def fpl_arr(arr, idx):
    return len(arr[idx])

# FILE NAME STRUCTURE
folder = '/hepgpu9-data1/sbridger/mphys_2d_network_2021-22/slice_arrays/new_registrations/'+organ+'/complete_arrays/'
prefix = 'COMPLETE_' + organ + '_'
tmp_img_arr=np.empty((image_array_length,512,512,4), dtype=data_type)
tmp_metric_arr=np.empty(metric_array_length, dtype=data_type)
tmp_reg_track=np.empty((rt_length,4), dtype=data_type)

multiple_patient_image_arrayT, multiple_hd_95_arrayT, multiple_hd_75_arrayT, multiple_hd_50_arrayT,\
multiple_dice_arrayT, multiple_rms_arrayT, multiple_msd_arrayT, multiple_msd_3d_arrayT, registration_trackT\
= tmp_img_arr, tmp_metric_arr, tmp_metric_arr,\
tmp_metric_arr, tmp_metric_arr, tmp_metric_arr, tmp_metric_arr, tmp_metric_arr, tmp_reg_track
print('here')
complete_img_name = folder+prefix+'multiple_patient_image_dvf_array.npy'
complete_dice_name = folder+prefix+'multiple_dice_array.npy'
complete_hd_95_name = folder+prefix+'multiple_hd_95_array.npy'
complete_hd_75_name = folder+prefix+'multiple_hd_75_array.npy'
complete_hd_50_name = folder+prefix+'multiple_hd_50_array.npy'
complete_rms_name = folder+prefix+'multiple_rms_array.npy'
complete_msd_name = folder+prefix+'multiple_msd_array.npy'
complete_reg_track_name = folder+prefix+'registration_track.npy'

names = [complete_img_name, complete_dice_name, complete_hd_95_name, complete_hd_75_name, complete_hd_50_name,\
        complete_rms_name, complete_msd_name, complete_reg_track_name]
print(names[0], names[1])
print(multiple_patient_image_arrayT.shape)
print(multiple_dice_arrayT.shape)
# SAVE THE FILES
np.save(complete_img_name, multiple_patient_image_arrayT)
np.save(complete_dice_name, multiple_dice_arrayT)
np.save(complete_hd_95_name, multiple_hd_95_arrayT)
np.save(complete_hd_75_name, multiple_hd_75_arrayT)
np.save(complete_hd_50_name, multiple_hd_50_arrayT)
np.save(complete_rms_name, multiple_rms_arrayT)
np.save(complete_msd_name, multiple_msd_arrayT)
np.save(complete_reg_track_name, registration_trackT)
print('done')

# OPEN THE SAVED FILE AS A MEMMAP
multiple_patient_image_array = np.lib.format.open_memmap(names[0], dtype=data_type, mode='r+', shape=tmp_img_arr.shape)
multiple_dice_array = np.lib.format.open_memmap(names[1], dtype=data_type, mode='r+', shape=tmp_metric_arr.shape)
multiple_hd_95_array = np.lib.format.open_memmap(names[2], dtype=data_type, mode='r+', shape=tmp_metric_arr.shape)
multiple_hd_75_array = np.lib.format.open_memmap(names[3], dtype=data_type, mode='r+', shape=tmp_metric_arr.shape)
multiple_hd_50_array = np.lib.format.open_memmap(names[4], dtype=data_type, mode='r+', shape=tmp_metric_arr.shape)
multiple_rms_array = np.lib.format.open_memmap(names[5], dtype=data_type, mode='r+', shape=tmp_metric_arr.shape)
multiple_msd_array = np.lib.format.open_memmap(names[6], dtype=data_type, mode='r+', shape=tmp_metric_arr.shape)
multiple_registration_track = np.lib.format.open_memmap(names[7], dtype=data_type, mode='r+', shape=(tmp_reg_track.shape))

# These functions concatenate the two seperate memmaped npy files into one file for
# the image and metric arrays 
def concatenate_memmap_metric_arrays(empty_array, value_array):
    first_arr=fpl_arr(value_array,0)
    second_arr=fpl_arr(value_array,1)
#     print(first_arr,second_arr)
    for n in range(first_arr):
        empty_array[n] = value_array[0][n]
    for n in range(second_arr):
        empty_array[n+first_arr] = value_array[1][n]

def concatenate_memmap_image_arrays(empty_array, value_array):
    first_arr=fpl_arr(value_array,0)
    second_arr=fpl_arr(value_array,1)
#     print(first_arr,second_arr)
    for n in range(first_arr):
        empty_array[n][:][:][:] = value_array[0][n][:][:][:]
        if n%100==0:
            print(n)
    for n in range(second_arr):
        empty_array[n+first_arr][:][:][:] = value_array[1][n][:][:][:] 
        if n%100==0:
            print(n)

In [ ]:
concatenate_memmap_image_arrays(multiple_patient_image_array, mpia)
concatenate_memmap_metric_arrays(multiple_dice_array, mda)
concatenate_memmap_metric_arrays(multiple_hd_95_array, mhd95)
concatenate_memmap_metric_arrays(multiple_hd_75_array, mhd75)
concatenate_memmap_metric_arrays(multiple_hd_50_array, mhd50)
concatenate_memmap_metric_arrays(multiple_rms_array, mrms)
concatenate_memmap_metric_arrays(multiple_msd_array, msd)
concatenate_memmap_metric_arrays(multiple_registration_track, rt)

# CHECK THE COPIED VALUES ARE THE SAME AS THE TRUE VALUES
# diffhd95, diffd, diffrms, diffmpia =[],[],[],[]
# upper_bound=6000; step=200
# range_choice=[*range(0,upper_bound,step)]
# if upper_bound < len(mda[0]):
#     choice=0
# else:
#     choice=1
    
# for i in range_choice:
#     diffhd95.append(multiple_hd_95_array[i]-mhd95[choice][i])
#     diffd.append(multiple_dice_array[i]-mda[choice][i])
#     diffrms.append(multiple_rms_array[i]-mrms[choice][i])
#     diffmpia.append(multiple_patient_image_array[i]-mpia[choice][i])
    
# print(np.max(diffmpia), np.max(diffmpia))
# print(np.max(diffrms), np.max(diffrms))
# print(np.max(diffhd95), np.max(diffhd95))
# print(np.max(diffd), np.max(diffd))

In [ ]:
# MEMMAP ARRAYS MUST BE FLUSHED TO SAVE THE ARRAYS TO DISK
multiple_patient_image_array.flush()
multiple_dice_array.flush()
multiple_hd_95_array.flush()
multiple_hd_75_array.flush()
multiple_hd_50_array.flush()
multiple_rms_array.flush()
multiple_msd_array.flush()
multiple_registration_track.flush()

In [ ]:
# OLD CODE
# LEFT IN CASE OF NEED TO USE

arr_names = [[multiple_patient_image_array, mpia],
            [multiple_dice_array, mda],
            [multiple_hd_95_array_3D, mhd953D],
            [msd_array_3D, mmsd3D]
            ]

# This loop concatenates the two seperate memmaped npy files into one file for
# the image and metric arrays 
for arr_name in arr_names:
    first_arr=fpl_arr(arr_name[1],0)
    second_arr=fpl_arr(arr_name[1],1)
#         print(first_arr,second_arr)
    for n in range(first_arr):
        if len(arr_name[1][0].shape)!=4: # if not image array
            arr_name[0][n] = arr_name[1][0][n]
        else:
            arr_name[0][n][:][:][:] = arr_name[1][0][n][:][:][:]
    for n in range(second_arr):
        if len(arr_name[1][1].shape)!=4: # if not image array
            arr_name[0][n+first_arr] = arr_name[1][1][n]
        else:
            arr_name[0][n+first_arr][:][:][:] = arr_name[1][1][n][:][:][:]